# Tutorial 8: Cuantificación de LLM y Efficent Fine-tunning.

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: María José Zambrano


### Objetivos del Tutorial

- Entender cuales son los problemas de almacenamiento de los LLM.
- Comprender que es la cuantificación de modelos.
- Como cargar un LLM cuantizado usando la librería de `bitandbytes`.
- Entender que es fine-tunning y su importancia. Conocer uno de lo algoritmo que ha optimizado este proceso.
- Revisar como realizar fine-tunning usando LoRA en BERT con la librería `peft`.

Los Large Language Models (LLMs) son conocidos por sus extensos requisitos computacionales. Típicamente, el tamaño de un modelo se calcula multiplicando el número de parámetros (tamaño) por la precisión de estos valores (tipo de dato). Sin embargo, para ahorrar memoria, los pesos pueden ser almacenados utilizando tipos de datos de menor precisión a través de un proceso conocido como cuantificación.

En la literatura se distinguen dos principales familias de técnicas de cuantificación de pesos:

- La Cuantificación Post-Entrenamiento (Post-Training Quantization, PTQ) es una técnica directa donde los pesos de un modelo ya entrenado se convierten a una precisión menor sin necesitar ningún reentrenamiento. Aunque es fácil de implementar, la PTQ se asocia con una posible degradación del rendimiento.
- El Entrenamiento Consciente de la Cuantificación (Quantization-Aware Training, QAT) incorpora el proceso de conversión de pesos durante la etapa de pre-entrenamiento o ajuste fino, lo que resulta en un rendimiento mejorado del modelo. Sin embargo, el QAT es computacionalmente costoso y exige datos de entrenamiento representativos.

Para trabajar con los LLMs cuantizados existe la librería `bitandbytes`, es una extensión para PyTorch que proporciona implementaciones eficientes de optimizadores de 4 y 8 bits. Su principal utilidad es mejorar la eficiencia del entrenamiento de modelos de deep learning, especialmente modelos grandes como los LLMs. Al usar bitsandbytes, se puede reducir el uso de memoria y acelerar las operaciones de entrenamiento sin una pérdida significativa de precisión. Esto es particularmente útil para entrenar modelos grandes en hardware con recursos limitados o para mejorar la velocidad de entrenamiento en hardware existente.

# Instalar librerías

En este tutorial, nos enfocaremos en utilizar optimizadores de 4 bits, utilizando la librerías `transformers` y `bitandbytes`

Primero necesitamos instalar las librerías necesarias:



In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Un ejemplo básico seria:

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

donde sólo es necesario definir con `True` el parámetro `load_in_4bit`. Si imprimimos el modelo podemos obtener más información.

In [3]:
print(model)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
       

Una vez cargado el modelo podemos hacer predicciones con el:

In [4]:
text = "Hello my name is"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

Hello my name is jimmy and I am a new member of the reddit clan. I am a new member of


## Uso avanzado

Primero, necesitas entender los diferentes argumentos que pueden ser ajustados y utilizados.

Todos estos parámetros pueden ser cambiados utilizando BitsandBytesConfig de transformers y pasándolo al argumento quantization_config al llamar a from_pretrained.

¡Hay que asegurarse de pasar load_in_4bit=True cuando uses BitsAndBytesConfig!

#### Cambio de dtype

El dtype se utiliza para cambiar el tipo de datos que se utilizará durante la computación. Por ejemplo, los estados ocultos podrían estar en float32, pero el cálculo puede configurarse en bf16 para acelerar el proceso.

Por defecto, el dtype de cálculo está configurado en float32.

In [5]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
model_cd_bf16 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [7]:
outputs = model_cd_bf16.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello my name is jimmy and I am a new member of the reddit clan. I am a new member of


#### Cambiar del tipo de cuantización

La integración de 4 bits viene con dos tipos diferentes de cuantización: FP4 y NF4. El tipo de datos NF4 significa Normal Float 4 y se introduce en el [artículo de QLoRA](https://arxiv.org/abs/2305.14314).

Puedes cambiar entre estos dos tipos de datos utilizando bnb_4bit_quant_type de BitsAndBytesConfig. Por defecto, se utiliza la cuantización FP4.

In [8]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [9]:
outputs = model_nf4.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello my name is John and I am a very happy man. I am a very happy man. I am a very


#### Uso de cuantización anidada para una inferencia y un entrenamiento más eficientes en memoria

Se aconseja utilizar la técnica de cuantización anidada. Esto ahorra más memoria sin pérdida de rendimiento adicional - según observaciones empíricas, esto permite el ajuste fino del modelo llama-13b en una NVIDIA-T4 de 16GB con una longitud de secuencia de 1024, tamaño de lote de 1 y pasos de acumulación de gradiente de 4.

Para habilitar esta característica, simplemente añade bnb_4bit_use_double_quant=True al crear tu configuración de cuantización.

In [10]:
from transformers import BitsAndBytesConfig

double_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)

model_double_quant = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=double_quant_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [11]:
outputs = model_double_quant.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello my name is jimmy and I am a new member of the reddit clan. I am a new member of


#### Poniendo al límite las capacidades de Google Colab


¿Hasta dónde podemos llegar con la cuantización? Veremos a continuación que es posible cargar un modelo a escala de 20B (40 GB en precisión media) completamente en la GPU utilizando este método de cuantización. 🤯

Carguemos el modelo con el tipo de cuantización NF4 para obtener mejores resultados, dtype de cálculo bfloat16, así como cuantización anidada para una carga de modelo más eficiente en memoria.

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

model-00044-of-00046.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

model-00045-of-00046.safetensors:   0%|          | 0.00/604M [00:00<?, ?B/s]

model-00046-of-00046.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

Hay que asegurarse si el modelo fue cargado en GPU:

In [13]:
model_4bit.hf_device_map

{'': 0}

Ahora podemos, ejecutar el modelo:

In [14]:
text = "Hello my name is"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model_4bit.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello my name is john i am a student i am a student i am a student i am a student i am


## ¿Qué es el Fine-tunning?

El Fine-tunning es una técnina ampliamente utilizada que permite la personalización de modelos de lenguaje preentrenados para tareas específicas. Este proceso requiere un conjunto de datos etiquetados para la completación de tareas, permitiendo que el modelo adapte sus pesos para alinearse de manera más efectiva con la tarea objetivo. Sin embargo, ajustar un modelo de lenguaje a gran escala desde cero puede ser un desafío en términos de los requisitos de GPU y el tiempo extenso que toma completar el entrenamiento. Es notable que hay LLM con hasta 7 mil millones de parámetros e incluso 1.76 billones de parámetros, como se ve en el caso de GPT-4. Por lo tanto, ajustar un LLM desde cero no siempre es la opción más eficiente.

### Como funciona LoRa

LORA surgió como solución cuando nos dimos cuenta de que el ajuste fino de un modelo de LLM desde cero requiere significativamente más parámetros que cuando se hace a partir de un modelo preentrenado. LORA ha demostrado que, para ajustar un modelo preentrenado, no es necesario modificar cada peso en cada capa. En cambio, introduce un método para aprender una representación de los pesos de la capa específica para la tarea en una dimensión más baja.

Desglosemos LORA paso a paso. Consideremos una capa completamente conectada con $m$ unidades de entrada y $n$ unidades de salida. La matriz de pesos para esta capa tiene dimensiones $m \times n$. Cuando proporcionamos una entrada $x$, la salida de esta capa se calcula utilizando la fórmula $Y = W X$.

Durante el fine-tunning con LORA, mantenemos $W$ fija e introducimos dos matrices, $A$ y $B$, en la ecuación. La nueva ecuación se convierte en $Y = W X + A \cdot B X$. Ahora, imagina que $m$ es 800 y $n$ es 3200, lo que da a la forma de $W$ 800 x 3200, resultando en 2.560.000 pesos.

Aquí es donde entra en juego la innovación de LORA. La matriz A tiene una forma de 800 x $r$, y la matriz B tiene una forma de $r$ x 3200. La clave es que puedes ajustar el valor de $r$. Si establecemos $r$ en 1, el número de pesos en esta capa se convierte en:

(800 x 1) + (1 x 3200) = 4000.

Esta es una reducción significativa en comparación con el ajuste fino con 2.560.000 pesos. En consecuencia, el fine-tunning de un LLM se vuelve mucho más rápido y requiere considerablemente menos recursos computacional, gracias al enfoque de LORA.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*d1ckUy_f3nfdTP_J0xzs-g.png">

Ahora pasemos al código y expliquemos cómo ajustar BERT para la clasificación de texto usando LoRa.

### Instalar librerías

In [1]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Descargar el dataset

Comenzamos obteniendo el conjunto de datos IMDB con dos categorías de Hugging Face. Este conjunto de datos está diseñado para la clasificación binaria de sentimientos y cuenta con un volumen de datos significativamente mayor en comparación con los conjuntos de datos de referencia anteriores. Dentro de este conjunto, disponemos de 25,000 reseñas de películas fuertemente polarizadas para el entrenamiento, junto con otras 25,000 para pruebas. Además, existe un conjunto adicional de datos no etiquetados que se pueden utilizar según sea necesario.

In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### Preprocesamiento

Se requiere para el procesamiento de texto requiere el uso de un tokenizador, junto con la implementación de una estrategia de relleno y truncamiento para gestionar de manera efectiva las longitudes de secuencias variables.


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Podemos crear un conjunto de datos más pequeño para el ajuste fino con el fin de ahorrar tiempo y simplificar el proceso.

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Inyectar LoRA al modelo BERT

A continuación, establecemos un objeto de configuración LORA utilizando los parámetros de PEFT de Hugging Face. El parámetro `task_Type` especifica el tipo de tarea para la cual el modelo será ajustado finamente. El parámetro $r$ denota las dimensiones de $A$ y $B$, como se mencionó anteriormente. Además, el `lora_alpha` actúa como un factor de escala, determinando la importancia relativa de los pesos en $A$ y $B$ en relación con los parámetros originales del modelo.

In [6]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [23]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Posteriormente, insertamos las matrices $A$ y $B$ en nuestro modelo invocando la función `get_peft_model`.

In [24]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

### Entrenamiento y evaluación del modelo

El Trainer no realiza de manera inherente una evaluación automática del rendimiento del modelo durante el proceso de entrenamiento. Para evaluar el modelo, tendremos que proporcionar al Trainer una función personalizada para calcular e informar métricas. Podemos utilizar la función 'evaluate.load' de la biblioteca Evaluate de Hugging Face, que ofrece una función de precisión sencilla para este propósito

In [25]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Luego llamamos a la función compute en la métrica para calcular la precisión de las predicciones del modelo. Antes de pasar las predicciones a compute, necesitamos convertir las predicciones en logit.

In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

A continuación, creamos una clase `TrainingArguments` que consolida un conjunto integral de hiperparámetros disponibles para personalización, junto con opciones de activación para diversas configuraciones de entrenamiento.

In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", report_to="none" ,
                                 num_train_epochs=25)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Finalmente entrenamos el modelo y observamos su rendimiento en cada época.

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.698534,0.468000
2,No log,0.687773,0.541000
3,No log,0.684066,0.523000
4,0.695000,0.678268,0.535000
5,0.695000,0.652691,0.653000
6,0.695000,0.627310,0.671000
7,0.695000,0.590901,0.701000
8,0.643000,0.549389,0.726000
9,0.643000,0.505386,0.748000
10,0.643000,0.466124,0.772000


TrainOutput(global_step=3125, training_loss=0.47622246826171877, metrics={'train_runtime': 2563.6204, 'train_samples_per_second': 9.752, 'train_steps_per_second': 1.219, 'total_flos': 6580725657600000.0, 'train_loss': 0.47622246826171877, 'epoch': 25.0})

In [30]:
# Ejemplo de texto para clasificación
text = "This movie was absolutely fantastic!"

# Tokenizar el texto
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [33]:
inputs.to('cuda:0')
outputs = model(**inputs)

# Obtener las probabilidades y la clase predicha
logits = outputs.logits
predicted_class = logits.argmax(dim=-1).item()

# Mostrar la clase predicha
print(f"Clase predicha: {predicted_class}")

Clase predicha: 1


In [35]:
# Ejemplo de texto para clasificación
text = "This movie was absolutely horrible!"

# Tokenizar el texto
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

inputs.to('cuda:0')
outputs = model(**inputs)

# Obtener las probabilidades y la clase predicha
logits = outputs.logits
predicted_class = logits.argmax(dim=-1).item()

# Mostrar la clase predicha
print(f"Clase predicha: {predicted_class}")

Clase predicha: 0


In [ ]:
model.save_pretrained("./lora_bert")
tokenizer.save_pretrained("./lora_bert")